In [ ]:
import math
import sys
sys.path.append('../../python/')  
from periphery import logicGate
from periphery import constant
print(constant.INV)

In [ ]:
class Precharger:
    def __init__(self, num_col, res_load, activity_col_write,
                 num_read_cell_op, num_write_cell_op, tech, input_param):
        self.initialized = False
        self.tech = tech
        self.input_param = input_param
        self.cap_output_BL
        self.initialize(num_col, res_load, activity_col_write,
                        num_read_cell_op, num_write_cell_op)

    def initialize(self, num_col, res_load, activity_col_write,
                   num_read_cell_op, num_write_cell_op):
        self.num_col = num_col
        self.res_load = res_load
        self.activity_col_write = activity_col_write
        self.num_read_cell_op = num_read_cell_op
        self.num_write_cell_op = num_write_cell_op
        # usually the pmos is large to pull up the bitline quickly
        self.width_pmos_precharger = 6 * self.tech['featureSize']
        self.width_pmos_equalizer = 1 * self.tech['featureSize']
        self.initialized = True

    def calculate__area(self, num_col, width_pmos_precharger, width_pmos_equalizer, feature_size,max_transistor_height, new_height, new_width, option):
    # Constants representing layout modification strategies
        MAGIC = 'MAGIC'
        OVERRIDE = 'OVERRIDE'
        NONE = 'NONE'

        # Step 1: Compute single gate areas
        h_pre, w_pre, _ = logicGate.calculate_logicgate_area(constant.INV, width_pmos_precharger, feature_size * max_transistor_height)
        h_eq, w_eq, _ = logicGate.calculate_logicgate_area(constant.INV, width_pmos_equalizer, feature_size * max_transistor_height)

        h_unit = h_pre + 2 * h_eq  # I don't know why we need 2 equalizers, but this is how it is in the original code
        w_unit = max(w_pre, w_eq)

        if new_width and option == 'NONE':
            if new_width < w_unit:
                raise ValueError("Precharger width is larger than assigned width.")

            num_unit_per_row = int(new_width // w_unit)
            num_unit_per_row = min(num_unit_per_row, num_col)
            num_row_unit = math.ceil(num_col / num_unit_per_row)

            width = new_width
            height = num_row_unit * h_unit
        else:
            width = num_col * w_unit
            height = h_unit

        area = height * width
        cap_output_BL_precharger = logicGate.calculate_logicgate_cap(constant.INV, 0 , width_pmos_precharger, feature_size * max_transistor_height)
        cap_output_BL_equalizer = logicGate.calculate_logicgate_cap(constant.INV, 0 , width_pmos_equalizer, feature_size * max_transistor_height)
        self.cap_output_BL = cap_output_BL_precharger + cap_output_BL_equalizer

    

    def calculate_latency(self,width_pmos, cap_load, res_load, num_read, num_write, temperature, tech, horowitz):
        # compute pull-up resistance
        res_pull_up = logicGate.calculate_on_resistance(width_pmos, constant.PMOS, temperature, tech)

        tau = res_pull_up * (cap_load + self.cap_output_BL) + res_load * cap_load / 2

        # conductance of PMOS
        gm = logicGate.calculate_transconductance(width_pmos, constant.PMOS, tech)

        # Horowitz model parameters beta
        beta = 1 / (res_pull_up * gm)

        # Horowitz calculation for delay
        ramp_output = None  
        base_latency = horowitz(tau, beta, 1e20, ramp_output)

        read_latency = base_latency * num_read
        write_latency = base_latency * num_write

        return read_latency, write_latency


    def calculate_power(self, cap_load, num_read, num_write, vdd, leakage_current):
        self.leakage = leakage_current * vdd * self.num_col
        min_read = min(self.num_read_cell_op, self.num_col)
        min_write = min(self.num_write_cell_op, self.num_col * self.activity_col_write)
        self.read_energy = cap_load * vdd**2 * min_read * 2 * num_read
        self.write_energy = cap_load * vdd**2 * min_write * num_write

